In [748]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV,BayesianRidge
import xgboost as xgb
from sklearn import metrics
import math
trainx = pd.read_csv("trainData.csv")
testx = pd.read_csv("testData.csv")
trainy = trainx['SalePrice']
truey = pd.read_csv("testy.csv")
del trainx['SalePrice']
del trainx['Id']
del testx['Id']
del trainx['GarageYrBlt']
del testx['GarageYrBlt']
data = pd.concat([trainx,testx])
data.index = range(1460)
data['MasVnrArea'] = data['MasVnrArea'].fillna(0)
data['LotFrontage'] = data['LotFrontage'].fillna(data['LotFrontage'].median())
data["MiscFeature"] = data["MiscFeature"].fillna("None")
data["Alley"] = data["Alley"].fillna("None")
data["Fence"] = data["Fence"].fillna("None")
data["BsmtCond"] = data["BsmtCond"].fillna("None")
data["MasVnrType"] = data["MasVnrType"].fillna("None")
data["NewDwelling"] = data["MSSubClass"].replace({20: 1, 30: 0, 40: 0, 45: 0,50: 0, 60: 1, 70: 0, 75: 0, 80: 0, \
                                                  85: 0,90: 0, 120: 1, 150: 0, 160: 0, 180: 0, 190: 0}) 
data['MSSubClass'] = data['MSSubClass'].astype(str)
data['GrLivArea'] = np.log1p(data['GrLivArea'])
data["PoolQC"] = data["PoolQC"].fillna("None")
data = data.replace({'Utilities': {'AllPub': 1, 'NoSeWa': 0, 'NoSewr': 0, 'ELO': 0},\
                    'ExterQual': {'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1},\
                    'BsmtCond': {'Ex': 5,'Gd': 4,'TA': 3,'Fa': 2,'Po': 1,'None': 0},\
                    'Functional': {'Typ': 0,'Min1': 1,'Min2': 1,'Mod': 2,'Maj1': 3,'Maj2': 4,'Sev': 5,'Sal': 6},\
                    'PavedDrive': {'Y': 1,'P': 0,'N': 0}})
data['Highseason'] = data["MoSold"].replace({1: 0, 2: 0, 3: 0, 4: 1, 5: 1, 6: 1, 7: 1, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0})
del data['MoSold']
data["BadHeating"] = data.HeatingQC.replace({'Ex': 0, 'Gd': 0, 'TA': 0, 'Fa': 1, 'Po': 1})
del data['HeatingQC']

In [749]:
overall_poor = data.OverallQual.copy()
overall_poor = 5 - overall_poor
overall_poor[overall_poor<0] = 0
data['overall_poor_quality'] = overall_poor
overall_good = data.OverallQual.copy()
overall_good = overall_good - 5
overall_good[overall_good<0] = 0
data['overall_good_quality'] = overall_good
del data['OverallQual']
overall_poor = data.OverallCond.copy()
overall_poor = 5 - overall_poor
overall_poor[overall_poor<0] = 0
data['overall_poor_condition'] = overall_poor
overall_good = data.OverallCond.copy()
overall_good = overall_good - 5
overall_good[overall_good<0] = 0
data['overall_good_condition'] = overall_good
del data['OverallCond']

In [736]:
#qual_dict = {None: 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}
#all_df["ExterQual"] = df["ExterQual"].map(qual_dict).astype(int)
#all_df["ExterCond"] = df["ExterCond"].map(qual_dict).astype(int)
#all_df["BsmtQual"] = df["BsmtQual"].map(qual_dict).astype(int)
#all_df["BsmtCond"] = df["BsmtCond"].map(qual_dict).astype(int)

In [737]:
print data.loc[:,data.isnull().any()].isnull().sum()
print data.dtypes[data.loc[:,data.isnull().any()].isnull().sum().index]

BsmtQual         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
Electrical        1
FireplaceQu     690
GarageType       81
GarageFinish     81
GarageQual       81
GarageCond       81
dtype: int64
BsmtQual        object
BsmtExposure    object
BsmtFinType1    object
BsmtFinType2    object
Electrical      object
FireplaceQu     object
GarageType      object
GarageFinish    object
GarageQual      object
GarageCond      object
dtype: object


In [750]:
cols = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
for col in cols:
    data[col] = data[col].fillna('None')
from sklearn.preprocessing import LabelEncoder
for c in ['GarageFinish', 'GarageQual', 'GarageCond']:
    lbl = LabelEncoder() 
    lbl.fit(list(data[c].values)) 
    data[c] = lbl.transform(list(data[c].values))
print data.loc[:,data.isnull().any()].isnull().sum()
print data.dtypes[data.loc[:,data.isnull().any()].isnull().sum().index]

BsmtQual         37
BsmtExposure     38
BsmtFinType1     37
BsmtFinType2     38
Electrical        1
FireplaceQu     690
dtype: int64
BsmtQual        object
BsmtExposure    object
BsmtFinType1    object
BsmtFinType2    object
Electrical      object
FireplaceQu     object
dtype: object


In [751]:
data = pd.get_dummies(data)
trainx = data[:1160]
testx = data[1160:]
testx.index = range(300)
trainy = np.log1p(trainy)
trainx = trainx.drop([518, 1154, 1157])
trainy = trainy.drop([518, 1154, 1157])
data.shape

(1460, 280)

In [752]:
model = RidgeCV(cv = 10).fit(trainx, trainy)
testy = model.predict(testx)
testy = np.exp(testy)-1
math.sqrt(metrics.mean_squared_error(testy, truey))

19314.89248503973